<a href="https://colab.research.google.com/github/Ashikuzzaman17/Bone-fracture/blob/main/First_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
ashikuzzamanshishir_bone_fracture_dataset_path = kagglehub.dataset_download('ashikuzzamanshishir/bone-fracture-dataset')

print('Data source import complete.')


Using Colab cache for faster access to the 'bone-fracture-dataset' dataset.
Data source import complete.


In [6]:
!pip install optuna

In [7]:
# ----------------------------------------------------------
# Cell 1 – Imports & global constants
# ----------------------------------------------------------
import os, random, time, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Deep‑learning
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, callbacks, regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.applications import (
    ResNet50, VGG16, InceptionV3, MobileNetV2, EfficientNetB0
)

# Transformers (timm)
import timm

# Image quality metrics
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# OpenCV for CLAHE
import cv2

# Misc
import optuna
import json
import psutil  # for memory usage


In [8]:
# ----------------------------------------------------------
# Cell 2 – Define dataset root and list sub‑folders
# ----------------------------------------------------------
DATASET_ROOT = Path('/kaggle/input/bone-fracture-dataset/Bone fracture dataset/Dataset')  # <-- change this

# Count sub‑folders (classes)
class_names = [d.name for d in DATASET_ROOT.iterdir() if d.is_dir()]
n_classes = len(class_names)
print(f'Found {n_classes} classes:', class_names)

# Count total images
total_imgs = sum(len([f for f in p.iterdir() if f.is_file()]) for p in DATASET_ROOT.glob('*'))
print(f'Total images: {total_imgs}')


Found 2 classes: ['normal', 'fracture']
Total images: 2127


In [9]:
# ----------------------------------------------------------
# Cell 3 – CLAHE + optional Gaussian blur + rescale
# ----------------------------------------------------------
def preprocess_img(img_path, target_size=(256,256)):
    """
    Load image, apply CLAHE, optional Gaussian blur,
    resize, and rescale to [0,1].
    """
    # Load
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # CLAHE on each channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = cv2.merge([clahe.apply(c) for c in cv2.split(img)])

    # Optional Gaussian blur
    # img = cv2.GaussianBlur(img, (3,3), 0)

    # Resize
    img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

    # Rescale to [0,1]
    img = img.astype(np.float32) / 255.0
    return img


In [ ]:
# ---------------- Cell 4 – PSNR / SSIM (Kaggle auto‑path) ----------------

import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from pathlib import Path

# --------------------------------------------------------------------------- #
# 1️⃣  Pre‑process an image: resize (optional), convert to RGB, normalise
#    (Using the preprocess_img from Cell 3)
# --------------------------------------------------------------------------- #
def preprocess_img(img_path, target_size=(256,256)):
    """
    Load image, apply CLAHE, optional Gaussian blur,
    resize, and rescale to [0,1].
    """
    # Load
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # CLAHE on each channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = cv2.merge([clahe.apply(c) for c in cv2.split(img)])

    # Optional Gaussian blur
    # img = cv2.GaussianBlur(img, (3,3), 0)

    # Resize
    img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

    # Rescale to [0,1]
    img = img.astype(np.float32) / 255.0
    return img


# --------------------------------------------------------------------------- #
# 2️⃣  PSNR & SSIM – auto‑resize processed image if shapes differ
# --------------------------------------------------------------------------- #
def psnr_ssim(original: np.ndarray,
              processed: np.ndarray,
              eps: float = 1e-10) -> tuple[float, float]:
    original = original.astype(np.float32)
    processed = processed.astype(np.float32)

    if original.shape != processed.shape:
        processed = cv2.resize(processed,
                               (original.shape[1], original.shape[0]),
                               interpolation=cv2.INTER_AREA)

    if original.shape != processed.shape:
        raise ValueError(f"Shape mismatch after resizing: {original.shape} vs {processed.shape}")

    # Ensure images are grayscale before calculating SSIM if they are not already
    if original.ndim == 3 and original.shape[-1] == 3:
      original_gray = cv2.cvtColor(original, cv2.COLOR_RGB2GRAY)
      processed_gray = cv2.cvtColor(processed, cv2.COLOR_RGB2GRAY)
    else:
      original_gray = original
      processed_gray = processed


    psnr_val = peak_signal_noise_ratio(original, processed, data_range=1.0)
    ssim_val = structural_similarity(original_gray,
                                      processed_gray,
                                      multichannel=False, # Set to False for grayscale
                                      data_range=1.0,
                                      eps=eps)
    return psnr_val, ssim_val

# --------------------------------------------------------------------------- #
# 3️⃣  Find first image folder under /kaggle/input (auto‑detect)
# --------------------------------------------------------------------------- #
def find_kaggle_image_folder() -> Path:
    """Return the first sub‑folder of /kaggle/input that contains images."""
    base = Path("/kaggle/input")
    if not base.is_dir():
        raise FileNotFoundError("Kaggle input directory not found")

    for candidate in base.iterdir():
        if candidate.is_dir():
            # quick check if it has any image files
            imgs = list(candidate.glob("*.jpg")) + \
                   list(candidate.glob("*.jpeg")) + \
                   list(candidate.glob("*.png"))
            if imgs:
                return candidate

    raise FileNotFoundError("No image folder found under /kaggle/input")

# --------------------------------------------------------------------------- #
# 4️⃣  Run PSNR/SSIM on all images in the detected folder
# --------------------------------------------------------------------------- #
def demo_folder(folder: Path) -> None:
    print(f"Scanning folder: {folder}")
    for img_file in folder.rglob("*"):
        if img_file.suffix.lower() not in {'.jpg', '.jpeg', '.png'}:
            continue

        # Raw image
        raw = cv2.imread(str(img_file))
        raw_rgb = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

        # Pre‑processed image using the function from Cell 3
        pre = preprocess_img(img_file,
                             target_size=(raw_rgb.shape[1], raw_rgb.shape[0]))

        psnr_val, ssim_val = psnr_ssim(raw_rgb, pre)
        print(f"{img_file.name:30} -> PSNR = {psnr_val:.2f} dB, SSIM = {ssim_val:.3f}")

# --------------------------------------------------------------------------- #
# 5️⃣  Execute the demo automatically (press *Run* to start)
# --------------------------------------------------------------------------- #
if __name__ == "__main__":
    # Use the known dataset path directly
    folder_path = Path('/kaggle/input/bone-fracture-dataset/Bone fracture dataset/Dataset')
    demo_folder(folder_path)

Scanning folder: /kaggle/input/bone-fracture-dataset/Bone fracture dataset/Dataset
379.png                        -> PSNR = 26.96 dB, SSIM = 0.954
340.png                        -> PSNR = 24.74 dB, SSIM = 0.935
48.png                         -> PSNR = 25.73 dB, SSIM = 0.946
227.png                        -> PSNR = 24.25 dB, SSIM = 0.938
61.png                         -> PSNR = 22.64 dB, SSIM = 0.915
377.png                        -> PSNR = 26.96 dB, SSIM = 0.954
222.png                        -> PSNR = 24.25 dB, SSIM = 0.938
37.png                         -> PSNR = 24.67 dB, SSIM = 0.925
35.png                         -> PSNR = 24.40 dB, SSIM = 0.967
384.png                        -> PSNR = 25.03 dB, SSIM = 0.880
376.png                        -> PSNR = 24.67 dB, SSIM = 0.973
133.png                        -> PSNR = 23.48 dB, SSIM = 0.955
70.png                         -> PSNR = 23.90 dB, SSIM = 0.934
73.png                         -> PSNR = 25.12 dB, SSIM = 0.903
92.png               

In [ ]:
# ----------------------------------------------------------
# Cell 5 – Split 80/10/10 with tf.data
# ----------------------------------------------------------
IMG_SIZE = (256, 256)
BATCH_SIZE = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_ROOT,
    validation_split=0.2,
    subset='training',
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_ROOT,
    validation_split=0.2,
    subset='validation',
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Split val_test into val and test (50/50)
val_ds = val_test_ds.take(int(0.5 * len(val_test_ds)))
test_ds = val_test_ds.skip(int(0.5 * len(val_test_ds)))

print('Training batches:', len(train_ds))
print('Validation batches:', len(val_ds))
print('Testing batches:', len(test_ds))


In [ ]:
# ----------------------------------------------------------
# Cell 6 – Data augmentation – two versions per image
# ----------------------------------------------------------
aug = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    fill_mode='nearest'
)

# Custom generator that yields 3 images per original: (orig, aug1, aug2)
def augment_generator(ds, augmentations=2):
    for batch, labels in ds:
        batch_aug = []
        for img in batch:
            batch_aug.append(img.numpy())   # original
            for _ in range(augmentations):
                aug_img = aug.random_transform(img.numpy())
                batch_aug.append(aug_img)
        # reshape to (batch_size*(1+aug), H, W, C)
        batch_aug = np.stack(batch_aug, axis=0)
        # Shuffle to mix originals with aug
        idx = np.arange(len(batch_aug))
        np.random.shuffle(idx)
        yield batch_aug[idx], np.repeat(labels.numpy(), augmentations+1, axis=0)

# Wrap in tf.data
train_aug_ds = tf.data.Dataset.from_generator(
    lambda: augment_generator(train_ds),
    output_types=(tf.float32, tf.float32),
    output_shapes=(
        (None, IMG_SIZE[0], IMG_SIZE[1], 3),
        (None, n_classes)
    )
).batch(BATCH_SIZE * (1+2)).prefetch(tf.data.AUTOTUNE)

# For validation and test we keep originals only
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)


In [ ]:
# ----------------------------------------------------------
# Cell 7 – Build a fine‑tuned CNN model
# ----------------------------------------------------------
def build_cnn(base_name='ResNet50'):
    """
    Returns a compiled model.
    base_name: one of ['ResNet50', 'VGG16', 'InceptionV3',
                       'MobileNetV2', 'EfficientNetB0']
    """
    base_model = {
        'ResNet50': ResNet50,
        'VGG16': VGG16,
        'InceptionV3': InceptionV3,
        'MobileNetV2': MobileNetV2,
        'EfficientNetB0': EfficientNetB0
    }[base_name](weights='imagenet',
                  include_top=False,
                  input_shape=(*IMG_SIZE, 3))
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.5, rate=0.5),
        layers.Dense(256, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-4)),
        layers.Dropout(0.5),
        layers.Dense(n_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy', Precision(name='prec'), Recall(name='rec')]
    )
    return model


In [ ]:
# ----------------------------------------------------------
# Cell 8 – Train multiple CNNs with early stopping
# ----------------------------------------------------------
cnn_models = ['ResNet50', 'VGG16', 'InceptionV3',
              'MobileNetV2', 'EfficientNetB0', 'ResNet50', 'VGG16']

history_dict = {}
train_start = time.time()

for name in cnn_models:
    print(f'\n=== Training {name} ===')
    model = build_cnn(name)

    es = callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss')
    csv = callbacks.CSVLogger(f'{name}_history.csv')

    history = model.fit(
        train_aug_ds,
        epochs=40,
        validation_data=val_ds,
        callbacks=[es, csv]
    )
    history_dict[name] = history.history
    model.save(f'{name}_final.h5')

    print(f'{name} training time: {time.time() - train_start:.1f}s')

print('All CNN training finished.')


In [ ]:
# ----------------------------------------------------------
# Cell 9 – Build a fine‑tuned transformer (timm)
# ----------------------------------------------------------
def build_transformer(model_name='vit_base_patch16_224', lr=1e-4):
    """
    Build a timm model for image classification.
    model_name: e.g., 'swin_tiny_patch4_window7_224', 'vit_tiny', 'levit_base', etc.
    """
    model = timm.create_model(
        model_name,
        pretrained=True,
        num_classes=n_classes,
        global_pool='avg'  # default
    )
    model.eval()

    # Freeze backbone
    for name, param in model.named_parameters():
        if 'head' not in name:   # freeze everything except classifier
            param.requires_grad = False

    # Wrap in Keras
    class TimmModel(tf.keras.Model):
        def __init__(self, timm_model):
            super().__init__()
            self.timm = timm_model
        def call(self, x):
            return self.timm(x)

    tf_model = TimmModel(model)

    tf_model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss='categorical_crossentropy',
        metrics=['accuracy', Precision(name='prec'), Recall(name='rec')]
    )
    return tf_model


In [ ]:
# ----------------------------------------------------------
# Cell 10 – Train a few transformer variants
# ----------------------------------------------------------
transformer_names = [
    'swin_tiny_patch4_window7_224',
    'vit_tiny_patch16_224',
    'levit_base',
    'deit_tiny_distilled_patch16_224',
    'mobilevit_s'
]

transform_history = {}
train_start = time.time()

for t_name in transformer_names:
    print(f'\n=== Training {t_name} ===')
    model = build_transformer(t_name, lr=1e-4)

    es = callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss')
    csv = callbacks.CSVLogger(f'{t_name}_history.csv')

    history = model.fit(
        train_aug_ds,
        epochs=40,
        validation_data=val_ds,
        callbacks=[es, csv]
    )
    transform_history[t_name] = history.history
    model.save(f'{t_name}_final.h5')

    print(f'{t_name} training time: {time.time() - train_start:.1f}s')

print('All transformer training finished.')


In [ ]:
# ----------------------------------------------------------
# Cell 11 – Add CBAM attention and re‑train selected models
# ----------------------------------------------------------
# CBAM from https://github.com/Jongchan/attention-module
# Here we illustrate it for ResNet50 as an example.

def cbam_module(input_feature, reduction_ratio=16):
    # Channel Attention
    channel = layers.GlobalAveragePooling2D()(input_feature)
    channel = layers.Dense(channel.shape[-1]//reduction_ratio,
                           activation='relu',
                           use_bias=True,
                           kernel_initializer='he_normal')(channel)
    channel = layers.Dense(channel.shape[-1],
                           activation='sigmoid',
                           use_bias=True,
                           kernel_initializer='he_normal')(channel)
    channel = layers.Reshape((1,1,channel.shape[-1]))(channel)
    channel_attention = layers.Multiply()([input_feature, channel])

    # Spatial Attention
    spatial = layers.Conv2D(1, kernel_size=7,
                            padding='same',
                            activation='sigmoid',
                            use_bias=False)(channel_attention)
    return layers.Multiply()([channel_attention, spatial])

# Example: Insert CBAM after base_model in CNN
def build_cnn_cbam(base_name='ResNet50'):
    base_model = {
        'ResNet50': ResNet50,
        'VGG16': VGG16,
        'InceptionV3': InceptionV3,
        'MobileNetV2': MobileNetV2,
        'EfficientNetB0': EfficientNetB0
    }[base_name](weights='imagenet',
                  include_top=False,
                  input_shape=(*IMG_SIZE, 3))
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.Lambda(lambda x: cbam_module(x)),
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-4)),
        layers.Dropout(0.5),
        layers.Dense(n_classes, activation='softmax')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Uncomment to re‑train one model with CBAM
# model_cbam = build_cnn_cbam('ResNet50')
# ... fit as above


In [ ]:
# ----------------------------------------------------------
# Cell 12 – Evaluate on test set
# ----------------------------------------------------------
def evaluate_model(model_path):
    model = tf.keras.models.load_model(model_path,
                                       custom_objects={'CBAM': cbam_module})   # if using CBAM
    loss, acc, prec, rec = model.evaluate(test_ds, verbose=0)
    print(f'{model_path} - Acc:{acc:.4f} Prec:{prec:.4f} Rec:{rec:.4f}')
    return acc, prec, rec

# Pick the best CNN (by val accuracy)
best_cnn_name = max(history_dict, key=lambda k: max(history_dict[k]['val_accuracy']))
print('\nBest CNN:', best_cnn_name)
evaluate_model(f'{best_cnn_name}_final.h5')

# Pick the best Transformer
best_tf_name = max(transform_history, key=lambda k: max(transform_history[k]['val_accuracy']))
print('\nBest Transformer:', best_tf_name)
evaluate_model(f'{best_tf_name}_final.h5')


In [ ]:
# ----------------------------------------------------------
# Cell 13 – Confusion matrix + classification report
# ----------------------------------------------------------
import sklearn.metrics as skm

def get_predictions(ds):
    preds = []
    truths = []
    for batch, labels in ds:
        probs = model.predict(batch)
        preds.extend(np.argmax(probs, axis=-1))
        truths.extend(np.argmax(labels, axis=-1))
    return np.array(preds), np.array(truths)

# Example for the best CNN
model = tf.keras.models.load_model(f'{best_cnn_name}_final.h5')
preds, truths = get_predictions(test_ds)

conf_mat = skm.confusion_matrix(truths, preds)
print('Confusion matrix:\n', conf_mat)

print('Classification report:')
print(skm.classification_report(truths, preds, target_names=train_ds.class_names))


In [ ]:
# ----------------------------------------------------------
# Cell 14 – Summary
# ----------------------------------------------------------
def layer_summary(model):
    model.build((None, *IMG_SIZE, 3))
    model.summary()
    trainable_count = np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])
    print('Trainable params:', trainable_count)

layer_summary(best_cnn_name)
layer_summary(best_tf_name)


In [ ]:
# ----------------------------------------------------------
# Cell 15 – Detailed metric plot (accuracy, precision, recall)
# ----------------------------------------------------------
import matplotlib.pyplot as plt

def plot_history(hist, title='Model History'):
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(hist['accuracy'], label='train')
    plt.plot(hist['val_accuracy'], label='val')
    plt.title(f'{title} Accuracy')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(hist['prec'], label='train')
    plt.plot(hist['val_prec'], label='val')
    plt.title(f'{title} Precision')
    plt.legend()
    plt.show()

for name, hist in history_dict.items():
    plot_history(hist, name)


In [ ]:
# ----------------------------------------------------------
# Cell 16 – Convert Keras H5 to ONNX (tf2onnx)
# ----------------------------------------------------------
!pip install tf2onnx

import tf2onnx

def export_to_onnx(h5_path, onnx_path):
    model = tf.keras.models.load_model(h5_path)
    spec = (tf.TensorSpec((None, *IMG_SIZE, 3), tf.float32, name="input"),)
    output_path = onnx_path
    model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec,
                                                output_path=output_path)
    print(f'Exported {h5_path} to {onnx_path}')

export_to_onnx(f'{best_cnn_name}_final.h5', f'{best_cnn_name}.onnx')
export_to_onnx(f'{best_tf_name}_final.h5', f'{best_tf_name}.onnx')


In [ ]:
# ----------------------------------------------------------
# Cell 17 – Create an HTML summary page
# ----------------------------------------------------------
summary = f"""
# Model Summary – {best_cnn_name} vs {best_tf_name}

| Metric | CNN ({best_cnn_name}) | Transformer ({best_tf_name}) |
|--------|----------------------|------------------------------|
"""

for metric in ['accuracy', 'prec', 'rec']:
    cnn_acc = max(history_dict[best_cnn_name]['val_accuracy'])
    tf_acc = max(transform_history[best_tf_name]['val_accuracy'])
    summary += f"| {metric} | {cnn_acc:.4f} | {tf_acc:.4f} |\n"

# Add confusion matrices and other notes
summary += "\n*See individual Jupyter cells for training history, PSNR/SSIM, and confusion matrices.*"

open('model_summary.md', 'w').write(summary)
print('Model summary written to model_summary.md')


In [ ]:
# ----------------------------------------------------------
# Cell 18 – Final timing & memory usage
# ----------------------------------------------------------
train_end = time.time()
print(f'Total training time: {train_end - train_start:.1f}s')

# Memory usage
import psutil
process = psutil.Process()
print('RAM used (MB):', process.memory_info().rss / (1024**2))
